In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.prompts import ChatPromptTemplate
from dotenv import (
    load_dotenv, 
    find_dotenv
)
import os

load_dotenv(
    find_dotenv()
)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")



def loadDataAndRetriever(data_path, **kwargs) :
    if kwargs["parse_function"] :
        print("[INFO] ----> Running the Data processing pipeline for HyDE RAG...")
        print("[INFO] ----> Please wait a while.....\n")

        collection_name = os.path.splitext(data_path)[0].split(os.sep)[1] + "-vectorstore"
        print("[INFO] ----> Creating the collection name with :- {}".format(collection_name))

        persist_directory_name = os.path.splitext(data_path)[0].split(os.sep)[1] + "-VDB"
        print("[INFO] ----> Creating the collection name with :- {}".format(persist_directory_name))


        print("[INFO] ----> Creating the Vectorstore....\n")
        documents = PyMuPDFLoader(
            data_path
        ).load()
        print("[INFO] ----> Total Pages Extracted are :- {}".format(len(documents)))
        

        splitted_documents = RecursiveCharacterTextSplitter(
            chunk_size=512, 
            chunk_overlap=128,
            length_function=len,
            is_separator_regex=False,
        ).split_documents(documents)
        print(
            "[INFO] ----> Total document chunks created are :- {}".format(len(splitted_documents))
        )

        print("\n[INFO] ----> Creating the Chroma DB Retriever......")
        vectorstore = Chroma.from_documents(
            documents=splitted_documents,
            collection_name=collection_name,
            embedding = kwargs["embeddings_model"], 
            persist_directory=persist_directory_name,
        )

        retriever = vectorstore.as_retriever(
            search_type="similarity", 
            search_kwargs={"k" : 5},
        )
        print("[INFO] ----> Chroma Retriever created successfully.....\n")
        return retriever, kwargs["embeddings_model"]
    else :
        print("[INFO] ----> Change parse_function to True for execution....")
        return None, None


def generateAnswer(input_question, **kwargs) :
    if kwargs["parse_function"] :

        user_question = input_question

        verbose = -1


        def formatDocuments(docs) :
            return "\n\n".join(doc.page_content for doc in docs)


        hyDETemplate = """You are an english expert mastering in creating hypothetical answers. 
        So, for the given user question generate a hypothetical answer. 
        Do not generate anything else just the answer. The question that you need to answer is: 
        Question: {user_question}
        """

        print("[INFO] ----> Creating the HyDE-Pipeline, please wait....")

        prompt = ChatPromptTemplate.from_template(hyDETemplate)
        user_question = prompt.format(
            user_question=USER_QUESTION,
        )

        hypothetical_answer = llm_groq.invoke(user_question).content

        if verbose > -1 :
            print(
                "[INFO] ----> The Hypothetical Answers for the:- \nQuestion:- {}\nHypo-Answer:- {}".format(user_question, hypothetical_answer)
            )

        similar_documents = retriever.invoke(
            hypothetical_answer
        )


        if verbose > -1 :
            for doc in similar_documents :
                print(doc.page_content)
                print("==" * 100)
        print("\n[INFO] ----> Pipeline created with HyDE.....")


        template = """You are an excellent assistant. 
        Answer the following question in a detailed manner based on the below provided context: 

        Context:- {context}\n
        Question:- {question}\n

        Always remember to provide a complete answer for the question asked.
        """

        prompt = ChatPromptTemplate.from_template(template)

        format_documents = formatDocuments(similar_documents)

        if verbose > -1 :
            print(
            "[INFO] ----> The context for the LLM is :- \nContext :- \n{}".format(format_documents)
        )

        print(
            "[INFO] ----> Crafting the answer for the question :- {}\n[INFO] ----> Please Wait.....".format(
                USER_QUESTION
                )
            )

        final_query = prompt.format(
            context=format_documents, 
            question=USER_QUESTION,
        )

        hyDEResponse = llm_google.invoke(
            final_query
        ).content

        print("\n[INFO] ----> Final Answer :- {}{}".format("\n", hyDEResponse))
        return hyDEResponse
    else :
        print("[INFO] ----> Change parse_function to True for execution....")
        return None
        
        
embeddings_google = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

llm_groq = ChatGroq(
    model="gemma2-9b-it", 
    temperature=0.3,
    max_tokens=512,
    model_kwargs={"top_p" : 0.9}
)

llm_openai = ChatOpenAI(
    model="gpt-4o", 
    max_tokens=1024, 
    temperature=0.3,
)

llm_google = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-001",
    temperature=0.2,
    max_tokens=1024,
)

retriever, embeddings = loadDataAndRetriever(
    data_path="Data\\ReAct.pdf",
    embeddings_model=embeddings_google,
    parse_function=True,
)


USER_QUESTION = "What is Chain of Thought prompting and how it is related to ReAct."
answer = generateAnswer(
    input_question=USER_QUESTION, 
    parse_function=True
)

[INFO] ----> Running the Data processing pipeline for HyDE RAG...
[INFO] ----> Please wait a while.....

[INFO] ----> Creating the collection name with :- ReAct-vectorstore
[INFO] ----> Creating the collection name with :- ReAct-VDB
[INFO] ----> Creating the Vectorstore....

[INFO] ----> Total Pages Extracted are :- 33
[INFO] ----> Total document chunks created are :- 293

[INFO] ----> Creating the Chroma DB Retriever......
[INFO] ----> Chroma Retriever created successfully.....



In [13]:
USER_QUESTION = "What is Chain of Thought prompting and how it is related to ReAct."
answer = generateAnswer(
    input_question=USER_QUESTION, 
    parse_function=True
)

[INFO] ----> Creating the HyDE-Pipeline, please wait....

[INFO] ----> Pipeline created with HyDE.....
[INFO] ----> Crafting the answer for the question :- What is Chain of Thought prompting and how it is related to ReAct.
[INFO] ----> Please Wait.....

[INFO] ----> Final Answer :- 
The provided context focuses on the ReAct framework, which combines reasoning and acting to enable agents to solve tasks in an environment. While the context doesn't explicitly mention Chain of Thought (CoT) prompting, it's important to understand how these concepts relate.

**Chain of Thought Prompting** is a technique used to improve the performance of large language models (LLMs) on reasoning tasks. It involves prompting the model to generate a step-by-step reasoning process, similar to how a human would think through a problem. This helps the model break down complex tasks into smaller, more manageable steps, leading to more accurate and explainable solutions.

**ReAct** is a framework that combines rea

[INFO] ----> Creating the HyDE-Pipeline, please wait....

[INFO] ----> Pipeline created with HyDE.....
[INFO] ----> Crafting the answer for the question :- What is Chain of Thought prompting and how it is related to ReAct.
[INFO] ----> Please Wait.....

[INFO] ----> Final Answer :- 
The provided context focuses on the ReAct paradigm, which combines reasoning and acting with language models for task solving. While the text doesn't explicitly mention "Chain of Thought" prompting, it's important to understand how these concepts relate.

**Chain of Thought Prompting** is a technique used to improve the reasoning abilities of language models. It involves prompting the model to generate a step-by-step reasoning process, often in the form of a series of intermediate thoughts, leading to the final answer. This helps the model break down complex problems into smaller, more manageable steps, improving its accuracy and transparency.

**ReAct** builds upon the idea of reasoning by incorporating ac